In [2]:
import numpy as np


def process_text(text: str) -> list[list[int, int, int]]:
    """Processes the text into a list of tuples of start and end coordinates

    Args:
        text (str): Input text from puzzle

    Returns:
        list: List of tuples of start and end coordinates
    """
    starts = []
    ends = []
    for line in text:
        start, end = line.split("~")
        starts.append([int(x) for x in start.split(",")])
        ends.append([int(x) for x in end.split(",")])
    processed = list(zip(starts, ends))
    processed.sort(key=lambda x: x[0][2])
    return processed


def create_arr(block_coordinates: list[list[int, int, int]]) -> np.array:
    """Creates an array of zeros with the correct dimensions to hold the blocks

    Args:
        block_coordinates (list[list[int, int, int]]): List of tuples of start and end coordinates

    Returns:
        np.array: Array of zeros with the correct dimensions to hold the blocks
    """
    arr = np.array(block_coordinates)
    xmax = arr[:, :, 0].max() + 1
    ymax = arr[:, :, 1].max() + 1
    zmax = arr[:, :, 2].max() + 1
    return np.zeros((zmax, ymax, xmax))


def initiate_block(
    coordinates: list[list[int, int, int]], count: int, arr: np.array
) -> None:
    """Initiates a block in the array

    Args:
        coordinates (list[list[int, int, int]]): List of tuples of start and end coordinates
        count (int): The number of the block
        arr (np.array): Array to hold the blocks
    """
    xs, ys, zs = coordinates[0]
    xe, ye, ze = coordinates[1]
    if (xs == xe) and (zs == ze):
        for i in range(ys, ye + 1):
            arr[(zs, i, xs)] = count
    elif (xs == xe) and (ys == ye):
        for i in range(zs, ze + 1):
            arr[(i, ys, xs)] = count
    elif (zs == ze) and (ys == ye):
        for i in range(xs, xe + 1):
            arr[(zs, ys, i)] = count


def drop_block(count: int, arr: np.array) -> None:
    """Drops a block in the array until it can't fall any further

    Args:
        count (int): The number of the block
        arr (np.array): Array to hold the blocks
    """
    can_fall = True
    while can_fall:
        block = np.where(arr == count)
        xs, ys, zs = block
        block = list(zip(xs, ys, zs))
        for z, y, x in block:
            if (z == 0) or (
                (arr[(z - 1, y, x)] != 0) and ((arr[(z - 1, y, x)] != count))
            ):
                can_fall = False
        if can_fall:
            for z, y, x in block:
                arr[(z - 1, y, x)] = count
                arr[(z, y, x)] = 0


def create_block_pile(text: str) -> np.array:
    """Creates a block pile from the input text

    Args:
        text (str): Input text from puzzle

    Returns:
        np.array: Array containing the block pile
    """
    puzzle_text = process_text(text)
    start_arr = create_arr(puzzle_text)
    for count, line in enumerate(puzzle_text):
        initiate_block(line, count + 1, start_arr)
        drop_block(count + 1, start_arr)
    return start_arr


def check_neighbours(block: int, arr: np.array) -> set:
    """Checks the neighbours of a block

    Args:
        block (int): The number of the block
        arr (np.array): Array containing the block pile

    Returns:
        set: Set of the blocks that the block is resting on
    """
    block_pos = np.where(arr == block)
    xs, ys, zs = block_pos
    block_pos = list(zip(xs, ys, zs))
    neighbours = set()
    for z, y, x in block_pos:
        if arr[(z - 1, y, x)] not in [0, block]:
            neighbours.add(arr[(z - 1, y, x)])
    return neighbours


def count_supports(arr: np.array) -> dict[set]:
    """Counts the number of blocks that each block is resting on

    Args:
        arr (np.array): Array containing the block pile

    Returns:
        dict[set]: Dictionary of the blocks that each block is resting on
    """
    resting_on = dict()
    for i in range(int(arr.max())):
        below = check_neighbours(i + 1, arr)
        resting_on[i + 1] = below
    return resting_on


def solve(text: str, part: int = 1) -> int:
    """Solves the puzzle

    Args:
        text (str): Input text from puzzle
        part (int, optional): Puzzle part. Defaults to 1.

    Returns:
        int: Answer to the puzzle
    """
    arr = create_block_pile(text)
    resting_on = count_supports(arr)
    if part == 1:
        supports = set()
        for below in resting_on.values():
            if len(below) == 1:
                supports = supports.union(below)
        return int(arr.max() - len(supports))
    if part == 2:
        moved = []
        for i in range(int(arr.max())):
            removed_blocks = set()
            removed_blocks.add(i + 1)
            moving = True
            while moving:
                moving = False
                for i in range(int(arr.max())):
                    moving_block = i + 1
                    if moving_block not in removed_blocks:
                        if len(resting_on[moving_block]) == 0:
                            pass
                        elif resting_on[moving_block].issubset(removed_blocks):
                            removed_blocks.add(moving_block)
                            moving = True
            moved.append(len(removed_blocks) - 1)

        return sum(moved)


input_text = [line.strip() for line in open("inputs/day22.txt", "r").readlines()]

print(f"Part 1 Answer: {solve(input_text)}")
print(f"Part 2 Answer: {solve(input_text, part=2)}")

Part 1 Answer: 461
Part 2 Answer: 74074
